# Optimized Plotly Dash Template

This notebook demonstrates an optimized implementation of a Plotly Dash dashboard with:
- Improved code structure and organization
- Performance optimizations
- Enhanced user interaction features
- Better styling

In [6]:
#!pip install flask_caching
import dash
from dash import dcc, html, callback
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from dash.exceptions import PreventUpdate
import numpy as np
from flask_caching import Cache

# Configure external stylesheets for better appearance
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [7]:
# Create a function to generate sample data (makes it reusable)
def generate_data():
    np.random.seed(42)  # For reproducibility
    months = ['January', 'February', 'March', 'April', 'May', 'June']
    sales = [10, 20, 30, 40, 50, 60]
    expenses = [8, 18, 28, 38, 48, 58]
    profit = [s - e for s, e in zip(sales, expenses)]
    
    return pd.DataFrame({
        'Month': months,
        'Sales': sales,
        'Expenses': expenses,
        'Profit': profit
    })

# Generate data once instead of recreating it on each callback
df = generate_data()

In [8]:
# Initialize the app with better configuration
app = dash.Dash(
    __name__,
    external_stylesheets=external_stylesheets,
    suppress_callback_exceptions=True,  # Useful for multi-page apps
    meta_tags=[
        {'name': 'viewport', 'content': 'width=device-width, initial-scale=1.0'}  # For mobile responsiveness
    ]
)

# Setup caching to improve performance
cache = Cache(
    app.server,
    config={
        'CACHE_TYPE': 'SimpleCache',
        'CACHE_DEFAULT_TIMEOUT': 300  # 5 minutes
    }
)

# Create reusable component functions for better organization
def create_header():
    return html.Div([
        html.H1('Sales and Expenses Dashboard', className='dashboard-title'),
        html.P('Interactive visualization of monthly business performance', className='dashboard-subtitle')
    ], className='header')

def create_controls():
    return html.Div([
        html.Div([
            html.Label('Select Metric:'),
            dcc.Dropdown(
                id='metric-selector',
                options=[
                    {'label': 'Sales', 'value': 'Sales'},
                    {'label': 'Expenses', 'value': 'Expenses'},
                    {'label': 'Profit', 'value': 'Profit'}
                ],
                value='Sales',
                clearable=False
            )
        ], className='dropdown-container'),
        
        html.Div([
            html.Label('Chart Type:'),
            dcc.RadioItems(
                id='chart-type',
                options=[
                    {'label': 'Line', 'value': 'line'},
                    {'label': 'Bar', 'value': 'bar'}
                ],
                value='line',
                labelStyle={'display': 'inline-block', 'margin-right': '10px'}
            )
        ], className='radio-container')
    ], className='controls-container')

In [9]:
# Improved app layout with better structure and components
app.layout = html.Div([
    create_header(),
    
    create_controls(),
    
    html.Div([
        dcc.Tabs([
            dcc.Tab(label='Chart View', children=[
                dcc.Loading(
                    id='loading-chart',
                    type='circle',
                    children=dcc.Graph(id='main-chart', config={'displayModeBar': True})
                )
            ]),
            dcc.Tab(label='Data View', children=[
                html.Div(id='data-table-container', className='table-container')
            ])
        ])
    ], className='tabs-container'),
    
    # Hidden div to store intermediate data
    html.Div(id='intermediate-data', style={'display': 'none'})
], className='dashboard-container')

# Add custom CSS to enhance the appearance
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>Business Analytics Dashboard</title>
        {%favicon%}
        {%css%}
        <style>
            body {
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                margin: 0;
                background-color: #f7f7f7;
            }
            .dashboard-container {
                max-width: 1200px;
                margin: 0 auto;
                padding: 20px;
            }
            .header {
                text-align: center;
                margin-bottom: 30px;
            }
            .dashboard-title {
                color: #2c3e50;
                margin-bottom: 5px;
            }
            .dashboard-subtitle {
                color: #7f8c8d;
                font-size: 1.1em;
            }
            .controls-container {
                display: flex;
                flex-wrap: wrap;
                gap: 20px;
                margin-bottom: 20px;
                padding: 15px;
                background-color: #ffffff;
                border-radius: 8px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            }
            .dropdown-container, .radio-container {
                flex: 1;
                min-width: 200px;
            }
            .tabs-container {
                background-color: #ffffff;
                border-radius: 8px;
                padding: 15px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            }
            .table-container {
                padding: 15px 0;
                overflow-x: auto;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

In [10]:
# Optimized callback with caching for better performance
@app.callback(
    Output('intermediate-data', 'children'),
    [Input('metric-selector', 'value')]
)
@cache.memoize()  # Cache the result to improve performance
def process_data(selected_metric):
    # This would typically involve heavier data processing
    # Since we cached it, expensive operations will only run once per input combination
    return selected_metric


@app.callback(
    Output('main-chart', 'figure'),
    [
        Input('chart-type', 'value'),
        Input('intermediate-data', 'children')
    ]
)
def update_chart(chart_type, selected_metric):
    if not selected_metric:
        raise PreventUpdate
    
    # Use an efficient approach to create the figure
    if chart_type == 'line':
        fig = px.line(
            df,
            x='Month',
            y=selected_metric,
            title=f'Monthly {selected_metric}',
            markers=True
        )
    else:  # bar chart
        fig = px.bar(
            df,
            x='Month',
            y=selected_metric,
            title=f'Monthly {selected_metric}',
        )
    
    # Add custom styling to the chart
    fig.update_layout(
        plot_bgcolor='#ffffff',
        paper_bgcolor='#ffffff',
        font={'color': '#2c3e50', 'family': 'Segoe UI'},
        margin=dict(l=40, r=40, t=50, b=40),
        hovermode='closest',
        xaxis=dict(
            showgrid=True,
            gridcolor='#f0f0f0',
            showline=True,
            linecolor='#e0e0e0'
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='#f0f0f0',
            showline=True,
            linecolor='#e0e0e0'
        )
    )
    
    # Add trending information
    values = df[selected_metric].values
    trend = 'increasing' if values[-1] > values[0] else 'decreasing'
    color = 'green' if trend == 'increasing' and selected_metric != 'Expenses' \
        else ('red' if trend == 'decreasing' and selected_metric != 'Expenses' \
        else ('red' if trend == 'increasing' and selected_metric == 'Expenses' \
        else 'green'))
        
    fig.add_annotation(
        text=f"Trend: {trend.capitalize()}",
        x=1,
        y=values[-1],
        xref="x domain",
        yref="y",
        showarrow=True,
        arrowhead=4,
        arrowsize=1,
        arrowcolor=color,
        ax=40,
        ay=-40,
        font=dict(
            size=12,
            color=color
        ),
        align="center"
    )
    
    return fig


@app.callback(
    Output('data-table-container', 'children'),
    [Input('intermediate-data', 'children')]
)
def update_table(selected_metric):
    if not selected_metric:
        raise PreventUpdate
    
    # Create an interactive data table
    from dash import dash_table
    
    return dash_table.DataTable(
        data=df.to_dict('records'),
        columns=[
            {'name': 'Month', 'id': 'Month'},
            {'name': selected_metric, 'id': selected_metric}
        ],
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'padding': '10px',
            'minWidth': '100px'
        },
        style_header={
            'backgroundColor': '#f2f2f2',
            'fontWeight': 'bold',
            'border': '1px solid #ddd'
        },
        style_data={
            'border': '1px solid #ddd'
        },
        style_data_conditional=[
            {
                'if': {'column_id': selected_metric, 'filter_query': '{' + selected_metric + '} > 30'},
                'backgroundColor': '#e6f7e6',  # Light green for high values
                'color': 'green'
            },
            {
                'if': {'column_id': selected_metric, 'filter_query': '{' + selected_metric + '} < 20'},
                'backgroundColor': '#ffeded',  # Light red for low values
                'color': 'darkred'
            }
        ]
    )

In [12]:
# Define a debugging function to inspect app performance
def analyze_callback_performance():
    # Add profiling for larger applications
    import time
    
    start_time = time.time()
    # Code to profile would go here
    elapsed_time = time.time() - start_time
    
    print(f"Callback executed in {elapsed_time:.4f} seconds")

# Run the app with more optimal settings
if __name__ == '__main__':
    # For production, you could use a WSGI server like gunicorn
    app.run(
        debug=False,  # Set to False in production
        port=8050,
        host='0.0.0.0',  # Makes it accessible on your network
        dev_tools_hot_reload=False  # Less overhead
    )